In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,decode_predictions,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping, Callback

import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [2]:
target_size = (224,224)
batch_size = 32
seed = 42

In [3]:
input_path = "../images_in_bins/"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    directory=input_path+"train/",
    target_size=target_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=True,
    seed=seed
)

valid_generator = valid_datagen.flow_from_directory(
    directory=input_path+"val/",
    target_size=target_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=True,
    seed=seed
)

test_generator = test_datagen.flow_from_directory(
    directory=input_path+"test/",
    target_size=target_size,
    color_mode="rgb",
    batch_size=1,
    class_mode='sparse',
    shuffle=False,
    seed=seed
)

Found 29192 images belonging to 316 classes.
Found 747 images belonging to 137 classes.
Found 781 images belonging to 147 classes.


In [4]:
base_model = MobileNetV2(input_shape=(224,224,3),weights='imagenet', include_top=False)

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)

In [6]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
opt=Adam()
model.compile(optimizer=opt, loss='mean_absolute_percentage_error')

In [7]:
model_name = "mob2fromdir"
history = Callback()
checkpoint = ModelCheckpoint(filepath="../models/{}.h5".format(model_name))
stopping = EarlyStopping(patience=3,restore_best_weights = True)

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=valid_generator.n//test_generator.batch_size

In [9]:
hist = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[history,checkpoint,stopping]
)

Epoch 1/10
118/912 [==>...........................] - ETA: 42:09 - loss: 94764112.0534

OSError: cannot identify image file '../images_in_bins/train/156/39486160_250951245749716_997686482992889856_n.jpg'

In [ ]:
model.evaluate_generator(generator=test_generator,steps=STEP_SIZE_TEST)

In [ ]:
Y_pred = model.predict_generator(test_generator,steps = int(NUMBER_OF_PHOTOS*test_size/batch_size))

In [ ]:
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(hist.history['loss'][0:],'r',linewidth=3.0)
plt.plot(hist.history['val_loss'][0:],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves for model {}'.format(model_name),fontsize=16)